@author: Geidyon M. Girma

On-policy SARSA

In [1]:
import gym
import time
import itertools
import random
import numpy as np
env = gym.make('MountainCar-v0')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [2]:
# discretization of the observation space

bins = (30, 30)

def discretize_observation(observation):
  """
  Discretizes the observation space.

  Args:
    observation: The observation to discretize.

  Returns:
    The discretized observation.
  """

  low = env.observation_space.low
  high = env.observation_space.high

  return tuple(np.digitize(observation[i], np.linspace(low[i], high[i], bins[i] + 1)) - 1 for i in range(2))

In [34]:
# the on-policy SARSA

#initialization
alpha = 0.1
gamma = 1
epsilon = 0.1

# formaulate the state space with every combination of the discritsized elements of the states
states = itertools.product(np.arange(bins[0]), np.arange(bins[1]))

q={}
policy = {}

for state in states:
  q[state] = np.zeros((env.action_space.n)) # initialize the action-value function

  # initializing the epsilon-greedy policy with respect to the action-value function
  # policy[state] = np.full(((env.action_space.n)),epsilon/env.action_space.n)
  # best_action = np.argmax(q[state])  # selecting the best action based on the action-value function
  # policy[state][best_action] += 1-epsilon


  policy[state] = np.full(((env.action_space.n)),epsilon/env.action_space.n)
  policy[state][np.random.randint(3)] += 1-epsilon



  # if state[1] < 15:
  #   policy[state][0] += 1 - epsilon
  # elif state[1] > 15:
  #   policy[state][2] += 1 - epsilon
  # else:
  #   policy[state][np.random.randint(0,3)] += 1 - epsilon

In [45]:
# iteration

episodes = 5e4
start_timer = time.time()
episode = 1
while episode < episodes:

  # reset the environment
  obs = env.reset()
  state = discretize_observation(obs) # discretize the observation
  action = np.random.choice(np.arange(env.action_space.n), p = policy[state]) # selecting an action
  # generating an episodes using a an epsilon greedy policy
  for t in range(1,202):
    obs, reward, done, info = env.step(action)
    next_state = discretize_observation(obs) # next state
    next_action = np.random.choice(np.arange(env.action_space.n), p = policy[state]) # next action
    q[state][action] += alpha * (reward + gamma * q[next_state][next_action] - q[state][action]) # update the action-value function

    # update the policy based on the action-value function
    best_action = np.argmax(q[state]) # selecting the best action based on the action-value function
    policy[state] = np.full(((env.action_space.n)),epsilon/env.action_space.n) # resetting the policy for the given state
    policy[state][best_action] += 1 - epsilon # setting the probability of the best action to 1

    state = next_state
    action = next_action

    if done:
      break


  episode += 1
  if episode % 1000 == 0:
    end_timer = time.time()
    timer = end_timer - start_timer
    elapsed_time_struct = time.gmtime(timer)
    formatted_time = time.strftime("%H:%M:%S", elapsed_time_struct)
    print("Episode: ",episode, " time: ", formatted_time)

  if episode == episodes:
    print("done!")


Episode:  1000  time:  00:00:30
Episode:  2000  time:  00:01:02
Episode:  3000  time:  00:01:32
Episode:  4000  time:  00:02:04
Episode:  5000  time:  00:02:36
Episode:  6000  time:  00:03:05
Episode:  7000  time:  00:03:37
Episode:  8000  time:  00:04:08
Episode:  9000  time:  00:04:37
Episode:  10000  time:  00:05:09
Episode:  11000  time:  00:05:44
Episode:  12000  time:  00:06:19
Episode:  13000  time:  00:06:53
Episode:  14000  time:  00:07:29
Episode:  15000  time:  00:08:06
Episode:  16000  time:  00:08:36
Episode:  17000  time:  00:09:14
Episode:  18000  time:  00:09:49
Episode:  19000  time:  00:10:22
Episode:  20000  time:  00:10:53
Episode:  21000  time:  00:11:26
Episode:  22000  time:  00:11:55
Episode:  23000  time:  00:12:28
Episode:  24000  time:  00:13:02
Episode:  25000  time:  00:13:39
Episode:  26000  time:  00:14:12
Episode:  27000  time:  00:14:42
Episode:  28000  time:  00:15:14
Episode:  29000  time:  00:15:44
Episode:  30000  time:  00:16:19
Episode:  32000  ti

In [53]:
steps_to_solution = []

for j in range(100):
  obs = env.reset()
  state = discretize_observation(obs)
  for i in range(1,50000):
    action = np.argmax(policy[state])
    # print("step",i, "reward: ", reward," state: ", state," action=",action)
    obs, reward, done, info = env.step(action)
    state = discretize_observation(obs)

    if done:
      #print ("done")
      steps_to_solution.append(i)
      break
print(steps_to_solution)
avg_step = np.mean(steps_to_solution)
print("Average steps to solution: ",avg_step)

# ipythondisplay.clear_output(wait=True)
env.close()

[129, 132, 153, 162, 153, 131, 130, 131, 127, 131, 130, 156, 161, 131, 127, 185, 161, 130, 157, 131, 132, 157, 128, 130, 127, 105, 184, 127, 129, 133, 129, 157, 129, 158, 96, 130, 104, 162, 132, 158, 132, 158, 153, 161, 185, 129, 128, 130, 130, 129, 130, 131, 133, 157, 131, 132, 130, 130, 130, 130, 164, 102, 125, 129, 157, 171, 130, 129, 131, 130, 132, 128, 129, 129, 130, 129, 129, 132, 128, 129, 155, 129, 130, 95, 161, 183, 130, 133, 132, 131, 125, 133, 185, 128, 130, 100, 130, 124, 133, 132]
Average steps to solution:  136.86


In [18]:
obs = env.reset()
state = discretize_observation(obs)
reward = 0
for i in range(50000):
  action = np.argmax(policy[state])
  print("step",i, "reward: ", reward," state: ", state," action=",action)
  obs, reward, done, info = env.step(action)

  state = discretize_observation(obs)

  if done:
    print ("done")
    break

# ipythondisplay.clear_output(wait=True)
env.close()
print("Iterations that were run:",i)

step 0 reward:  0  state:  (11, 15)  action= 2
step 1 reward:  -1.0  state:  (11, 15)  action= 2
step 2 reward:  -1.0  state:  (11, 15)  action= 2
step 3 reward:  -1.0  state:  (11, 15)  action= 2
step 4 reward:  -1.0  state:  (11, 15)  action= 2
step 5 reward:  -1.0  state:  (11, 15)  action= 2
step 6 reward:  -1.0  state:  (11, 16)  action= 2
step 7 reward:  -1.0  state:  (11, 16)  action= 2
step 8 reward:  -1.0  state:  (12, 16)  action= 2
step 9 reward:  -1.0  state:  (12, 16)  action= 2
step 10 reward:  -1.0  state:  (12, 16)  action= 2
step 11 reward:  -1.0  state:  (12, 16)  action= 2
step 12 reward:  -1.0  state:  (12, 16)  action= 2
step 13 reward:  -1.0  state:  (12, 16)  action= 2
step 14 reward:  -1.0  state:  (12, 17)  action= 2
step 15 reward:  -1.0  state:  (13, 17)  action= 2
step 16 reward:  -1.0  state:  (13, 17)  action= 2
step 17 reward:  -1.0  state:  (13, 17)  action= 2
step 18 reward:  -1.0  state:  (13, 17)  action= 2
step 19 reward:  -1.0  state:  (13, 17)  act